<a href="https://colab.research.google.com/github/gaurav12899/models/blob/master/cryptoPred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from sklearn import preprocessing
from collections import deque
import random 
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.compat.v1.keras.layers import CuDNNLSTM 
SEQ_LEN=60
FUTURE_PERIOD_PREDICT = 3
RATIO_TO_PREDICT= 'LTC-USD'
EPOCHS=10
BATCH_SIZE=64
NAME= f"{RATIO_TO_PREDICT}-{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"

ratios=["BCH-USD","BTC-USD","ETH-USD","LTC-USD"]
main_df=pd.DataFrame()

def classify(current,future):
  if float(future)> float(current):
    return 1
  else:
    return 0


def preprocessing_df(df):
  df = df.drop("future",1)
  
  for col  in df.columns:
    if col != "target":
      df[col] = df[col].pct_change()
      df.dropna(inplace=True)
      df[col]=  preprocessing.scale(df[col].values)
      
    df.dropna(inplace=True)

  sequential_data=[]
  prev_days= deque(maxlen=SEQ_LEN)
  # print(df.head)

  # for col in df.columns:
  #   print(col)
  for i in df.values:
    prev_days.append([n for n in i[:-1]])
    if(len(prev_days)==SEQ_LEN):
      sequential_data.append([np.array(prev_days),i[-1]])

  random.shuffle(sequential_data)  
  # print(sequential_data)    

  buys=[]
  sells=[]
  for seq, target in sequential_data:
    if target==0:
      sells.append([seq, target])
    elif target==1:
      buys.append([seq, target])
  random.shuffle(buys)
  random.shuffle(sells)

  lower= min(len(buys), len(sells))

  sells=sells[:lower]
  buys= buys[:lower]

  sequential_data =buys+ sells
  random.shuffle(sequential_data)

  x=[]
  y=[]
  for seq, target in sequential_data:
    x.append(seq)
    y.append(target)

  return np.array(x),y  




for ratio in ratios:
  dataset="/content/drive/My Drive/crypto_prediction/crypto_data/crypto_data/{}.csv".format(ratio)
  
  df=pd.read_csv(dataset,names=["time","low","high","open","close","volume"])
  df.set_index("time",inplace=True)

  df.rename(columns={"close":"{}_close".format(ratio),"volume":"{}_volume".format(ratio)}, inplace=True )
  
  df= df[["{}_close".format(ratio),"{}_volume".format(ratio)]]
  
  if(len(main_df)==0):
    main_df=df
  else:
    main_df= main_df.join(df)
    

main_df["future"] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT)

main_df["target"]=list(map(classify, main_df[f"{RATIO_TO_PREDICT}_close"], main_df["future"]))
# print(main_df[[f"{RATIO_TO_PREDICT}_close", 'future', "target"]].head(10))


times=sorted(main_df.index.values)
last_5_perc = times[-int(.05*len(times))]
# print(last_5_perc)


validation_main_df=main_df[(main_df.index>= last_5_perc)]
main_df=main_df[(main_df.index< last_5_perc)]



train_x,train_y=preprocessing_df(main_df)
validation_x,validation_y =  preprocessing_df(validation_main_df)

print(f"train data: {len(train_x)} validation data:{len(validation_x)} " )
print(f"Don't buys: {train_y.count(0)}, buys: {train_y.count(1)}")
print(f"VALIDATION DON'T BUYS:{validation_y.count(0)}, VALIDATION BUYS:{validation_y.count(1)}")


model= Sequential()
model.add(CuDNNLSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(.2))
model.add(BatchNormalization())

model.add(CuDNNLSTM(128, return_sequences=True))
model.add(Dropout(.2))
model.add(BatchNormalization())

model.add(CuDNNLSTM(128))
model.add(Dropout(.2))
model.add(BatchNormalization())


model.add(Dense(32, activation='relu'))
model.add(Dropout(.2))


model.add(Dense(2, activation='softmax'))
model.add(Dropout(.2))

opt= tf.keras.optimizers.Adam(lr=1e-3, decay= 1e-5)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])
tensorboard= TensorBoard(log_dir=f"/content/drive/My Drive/crypto_prediction/logs/{NAME}")

filepath = "RNN_Final-{epoch:02d}-{val_accuracy:.3f}"  # unique file name that will include the epoch and the validation acc for that epoch
checkpoint = ModelCheckpoint("/content/drive/My Drive/crypto_prediction/models/{}.model".format(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')) # saves only the best ones



train_y=np.array(train_y)
validation_y=np.array(validation_y)

history= model.fit(train_x, train_y ,
                   batch_size=BATCH_SIZE,
                   epochs=EPOCHS, 
                   validation_data=(validation_x, validation_y),callbacks=[tensorboard, checkpoint])

# score = model.evaluate(validation_x, validation_y, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])
# # Save model
# model.save("/content/drive/My Drive/crypto_prediction/models/{}".format(NAME))

train data: 69400 validation data:3688 
Don't buys: 34700, buys: 34700
VALIDATION DON'T BUYS:1844, VALIDATION BUYS:1844
Epoch 1/10
1085/1085 [==============================] - ETA: 0s - loss: 2.9996 - accuracy: 0.5122

ValueError: ignored

In [0]:
len(train_y)

69400